<a href="https://colab.research.google.com/github/thoadao0301/AI-Project/blob/PhuongNDU/B%E1%BA%A3n_sao_c%E1%BB%A7a_Set_up_output_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install keras_facenet

     |████████████████████████████████| 2.3MB 3.8MB/s 
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-cp37-none-any.whl size=10386 sha256=9390b5229bf569f80ad9a03c4611f1241c70217eea0bdbb2c79034d4f1706c44
  Stored in directory: /root/.cache/pip/wheels/f6/53/9a/36c4b52fd22faf4f710d5047d874655b85a1b2cf77accfb9bd
Successfully built keras-facenet


In [9]:
import sys
import os, random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm
from keras_facenet import FaceNet
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

# **Training classification**

---



## Prepare dataset

In [19]:
# Gets a detection dict for each face
# in an image. Each one has the bounding box and
# face landmarks (from mtcnn.MTCNN) along with
# the embedding from FaceNet.
# return List of embedded images in folder
def get_embedding(dir, embedder):
    x = []
    with tqdm(total=len(os.listdir(dir)), file=sys.stdout) as pbar:
      for filename in os.listdir(dir):
        # set up file path
        filepath = os.path.join(dir,filename) 
        # load image from file
        image = Image.open(filepath)
        # convert to RGB, if needed
        image = image.convert('RGB')
        # convert to array
        pixels = np.asarray(image)
        # get embeddings for the faces in an image
        detections = embedder.extract(pixels, threshold=0.95)
        if(len(detections)<1):
            print('ERROR: Can\'t not detect face: ',filepath)
            continue
        if(len(detections)>1):
            print('ERROR: More than two face detected: ', filepath)
            continue
        x.append(detections[0]['embedding'])
        pbar.update(1)
    return x


In [20]:
# Get embedded list images and list labels in folder
def load_dataset(dir, embedder):
    x , y = [],[]
    for subdir in os.listdir(dir):
        subdir_path = dir+'/'+subdir
        
        data_embedding = get_embedding(subdir_path, embedder)
        for data in data_embedding:
            x.append(data)
            y.append(subdir)
    return x, y

In [21]:
# Create embedder tools with keras_facenet
embedder = FaceNet()

In [22]:
# Create train dataset
dataset_train_path= '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male' # Train data path
train_x, train_y = load_dataset(dataset_train_path, embedder)

100%|██████████| 188/188 [05:35<00:00,  1.78s/it]


In [ ]:
# Create test dataset
dataset_test_path= '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male_Test' # Test data path
test_x, test_y = load_dataset(dataset_test_path, embedder)

 47%|████▋     | 14/30 [00:14<00:16,  1.00s/it]

## Training model classification using SVM method

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)

In [ ]:
# tranning model
model = SVC(kernel='linear', probability=True)
model.fit(train_x, train_y)
# score
score_train = model.score(train_x, train_y)
score_test = model.score(test_x, test_y)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

## Use model to recognize faces in images (test for show image and probrability)

In [ ]:
# Get predict of image
import cv2
# read image from file path
image = cv2.imread('') # Image test path
# convert to RGB, if needed
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# convert to array
pixels = asarray(image)
# create the detector, using default weights
detections = embedder.extract(pixels)
# loop through each face in detections
for detect in detections:
    # get embedded image
    embedded_img = detect['embedding']

    # expand dimension for feeding data to the model
    samples = np.expand_dims(embedded_img, axis=0)

    # get predict from model
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)

    # get label name
    class_index = yhat_class[0]
    predict_names = out_encoder.inverse_transform(yhat_class)[0]

    # get probability
    class_probability = round(yhat_prob[0,class_index] * 100)
    print('Predict image has person: ', predict_names, ' with confidence:', class_probability)

# OUTPUT


In [ ]:
input_loc = '/content/gdrive/MyDrive/data/xxxxx.mp4'
output_loc = '/content/gdrive/MyDrive/data/'
my_file = open(output_loc + 'list_faces.txt','a+')

In [ ]:
def detect_face(image,frame_number,fps):
  # convert to array
  pixels = asarray(image)
  # create the detector, using default weights
  detections = embedder.extract(pixels)
  # loop through each face in detections
  for detect in detections:
    # get embedded image
    embedded_img = detect['embedding']
    # get coordinates (x,y) and weight, height (w, h) of the bounding box
    x,y,w,h = detect['box']
    
    # expand dimension for feeding data to the model
    samples = np.expand_dims(embedded_img, axis=0)

    # get predict from model
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)

    # get label name
    class_index = yhat_class[0]
    predict_names = out_encoder.inverse_transform(yhat_class)[0]

    # get probability
    class_probability = round(yhat_prob[0,class_index] * 100)
    
    # Put label name if probability >50% and 'Unknown'' if <= 50%
    if class_probability > 50:
      # Vẽ một hình tứ giác xung quanh các khuôn mặt phát hiện được. Vẽ trên ảnh màu.
      cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
      cv2.putText(image, "{}:{:.0f}%".format(predict_names,class_probability), 
                  (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
      # Hiển thị kết quả ra màn hình
      cv2_imshow('Face Detection', image)
      #time    
      duration= frame_number/fps
      minutes = int(duration/60)
      seconds = duration%60

      #write out
      my_file.write('Time('+ str(minutes) + ':' + str(seconds) + '),' 
                    + "{}:{:.0f}%".format(predict_names,class_probability)
      cv2.imwrite(output_loc + "frame/%#05d.jpg" % (frame_number), image)


In [ ]:
import cv2
cap = cv2.VideoCapture(input_loc)
video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
fps = cap.get(cv2.CAP_PROP_FPS)  
while cap.isOpened():
  # do stuff
  frame_number += 5
  if (frame_number > (video_length-1)):
    # Release the feed
    cap.release()
    cv2.destroyAllWindows()
    break

  cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
  # Extract the frame
  ret, frame = cap.read()
  # convert to RGB, if needed
  image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  # Thực thi Face Detection
  detect_face(image,frame_number,fps)
  
my_file.close()